In [1]:
from itertools import product

import msprime
import numpy as np
import tskit

In [2]:
sequence_length = 11
seed = 4
Ne = 10_000
mu = 2e-8
r = 1.25e-5
ts = msprime.sim_ancestry(
    samples=100,
    population_size=Ne,
    recombination_rate=r,
    sequence_length=sequence_length,
    random_seed=seed,
)
ts = msprime.sim_mutations(ts, rate=2e-5)

In [3]:
tests = [
    ("site", "D2_ij", "equal"),
    ("branch", "D2_ij", "equal"),
    ("site", "r2_ij", "isclose_nan"),
    ("branch", "r2_ij", "isclose"),
]

In [4]:
eq_funcs = {
    "equal": lambda a, b: (a == b).all(),
    "isclose": lambda a, b: np.isclose(a, b).all(),
    "isclose_nan": lambda a, b: np.isclose(a, b, equal_nan=True).all(),
}

In [5]:
sample_sets = [ts.samples(), ts.samples(), ts.samples(), ts.samples()]
ss_range = range(len(sample_sets))
indexes = list(product(ss_range, ss_range))
for mode, stat, comp in tests:
    print(f"mode={mode}, stat={stat}, comp={comp}")
    a = ts.ld_matrix_two_way(
        sample_sets,
        mode="site",
        stat="D2_ij",
        indexes=indexes,
    )
    b = ts.ld_matrix(mode="site", stat="D2", sample_sets=sample_sets)
    for i in ss_range:
        for j in range(len(indexes)):
            assert eq_funcs[comp](a[i], b[i])
    print("all equal")

mode=site, stat=D2_ij, comp=equal
all equal
mode=branch, stat=D2_ij, comp=equal
all equal
mode=site, stat=r2_ij, comp=isclose_nan
all equal
mode=branch, stat=r2_ij, comp=isclose
all equal
